# 1st Homology Inference Algorithm Comparison

- **Objective**: To validate and compare three algorithms (*Persistent Homology, $k$-NN Graph, Hodge Laplacian*) for inferring the first Betti number $\beta_1$ from a point cloud sampled from a manifold.
The comparison will be based on:
    - **Accuracy**;
    - **Robustness to parameters**;
    - **Consistency of the number of samples**;
    - **Computational efficiency**

## 1. Data generation

- Target manifold: Parametrized torus $\mathbb{T}$ in $\mathbb{R}^4$.
- The number of samples: 30, 300, 3000, 30000

In [5]:
import numpy as np

n_samples = 30

def generate_grid_points(n_samples: int) -> tuple[np.ndarray, np.ndarray]:
    thicks = np.linspace(0, 2 * np.pi, n_samples)
    u, v = np.meshgrid(thicks, thicks)
    return u.flatten(), v.flatten()

def generate_torus(u: np.ndarray,
                   v: np.ndarray) -> np.ndarray:
    return np.array([np.cos(u), np.sin(u), np.cos(v), np.sin(v)]).T


point_cloud = generate_torus(*generate_grid_points(n_samples))
print(point_cloud.shape)  # (n_samples**2, 4)

(900, 4)


## 2. Algorithms





### 2.1 Hodge Laplacian

In [10]:
from sklearn.metrics.pairwise import euclidean_distances

def compute_distance_matrix(point_cloud: np.ndarray) -> np.ndarray:
    return euclidean_distances(point_cloud)

def compute_edge_weights(distance_matrix: np.ndarray, epsilon: float) -> np.ndarray:
    return np.exp(-distance_matrix**2 / (4 * epsilon))

def compute_face_weights(edge_weights: np.ndarray) -> np.ndarray:
    T1 = np.einsum('ij,ik->ijk', edge_weights, edge_weights)   # k_ij * k_ik
    T2 = np.einsum('ij,jk->ijk', edge_weights, edge_weights)   # k_ij * k_jk
    T3 = np.einsum('ik,jk->ijk', edge_weights, edge_weights)   # k_ik * k_jk
    triangle_weights_tensor = (T1 + T2 + T3) / 3.0
    return triangle_weights_tensor


distance_matrix = compute_distance_matrix(point_cloud)
edge_weights = compute_edge_weights(distance_matrix, epsilon=0.1)
face_weights = compute_face_weights(edge_weights)

print(edge_weights.shape)  # (n_samples**2, n_samples**2)
print(face_weights.shape)  # (n_samples**2, n_samples**2, n_samples**2)

(900, 900)
(900, 900, 900)


In [ ]:
from scipy.sparse import lil_matrix, diags

def build_boundary_matrices(n_points: int, edge_weights: np.ndarray, face_weights: np.ndarray) -> tuple[lil_matrix, lil_matrix]:
    # Boundary matrix from edges to points
    B1 = lil_matrix((n_points, n_points * n_points))
    for i in range(n_points):
        for j in range(n_points):
            if i != j:
                idx = i * n_points + j
                B1[i, idx] = -1
                B1[j, idx] = 1

    # Boundary matrix from faces to edges
    B2 = lil_matrix((n_points * n_points, n_points * n_points * n_points))
    for i in range(n_points):
        for j in range(n_points):
            for k in range(n_points):
                if i != j and j != k and i != k:
                    idx = i * n_points * n_points + j * n_points + k
                    edge_ij = i * n_points + j
                    edge_ik = i * n_points + k
                    edge_jk = j * n_points + k
                    B2[edge_ij, idx] = face_weights[i, j, k]
                    B2[edge_ik, idx] = -face_weights[i, j, k]
                    B2[edge_jk, idx] = face_weights[i, j, k]

    return B1.tocsr(), B2.tocsr()


In [8]:
distance_matrix.shape

(900, 900)